## 운동 분류

In [1]:
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import matplotlib.pyplot as plt
from gcn_source import GraphRNNClassifier
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import cv2
import pickle

## data 

In [2]:
with open("start_dataloader.pkl", "rb") as f:
    saved_data = pickle.load(f)

loaded_dataset = saved_data["dataset"]
start_dataloader = DataLoader(
    loaded_dataset,
    batch_size=saved_data["batch_size"],
    shuffle=saved_data["shuffle"],
)
for inputs, labels in start_dataloader:
    print(inputs.shape)  # 모델의 입력 크기 확인
    break


torch.Size([4, 15, 1, 640, 480])


In [3]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
from cnn_source import Simple3DCNN
model = Simple3DCNN().to(device)
optimizer = optim.RAdam(model.parameters(),lr = 1e-3)
criterion = nn.CrossEntropyLoss()
scheduler = CosineAnnealingLR(optimizer, T_max=25, eta_min=0.0001)
def saveModel():
    torch.save(model.state_dict(), f'model/classifying.pt')
loss_ = []
valoss_ = []
logger = {"train_loss": list(),
          "validation_loss": list(),

          }

device

device(type='cuda', index=0)

In [4]:
def training(epochs) :

    for epoch in range(epochs):
        running_train_loss = 0.0
        running_vall_loss = 0.0
        total = 0
        model.train()
        for data in train_x:
            inputs = data
            outputs = data

            optimizer.zero_grad()  # zero the parameter gradients
            encoded, predicted_outputs = model((inputs))  # predict output from the model
            train_loss = criterion(predicted_outputs, outputs)# calculate loss for the predicted output

            train_loss.backward()  # backpropagate the loss
            optimizer.step()  # adjust parameters based on the calculated gradients
            running_train_loss += train_loss.item()  # track the loss value

        loss_.append(running_train_loss / n)
        with torch.no_grad():
            model.eval()
            for data in valid_x:
                inputs = data
                outputs = data

                encoded, predicted_outputs = model((inputs))
                val_loss = criterion(predicted_outputs, outputs)

                # The label with the highest value will be our prediction
                _, predicted = torch.max(predicted_outputs, 1)
                running_vall_loss += val_loss.item()
                total += outputs.size(0)
                val_loss_value = running_vall_loss / len(valid_x)


        valoss_.append(val_loss_value)

        avgtrainloss = np.mean(loss_)
        avgvalidloss = np.mean(valoss_)
        print('epoch', epoch + 1)
        print(f'train loss : {avgtrainloss}, validation loss : {avgvalidloss}')
        early_stopping(avgvalidloss, model)
        if early_stopping.early_stop:  # 조건 만족 시 조기 종료
            break
    saveModel()

## 모든 데이터 train

In [4]:
model.train()  # 학습 모드 설정
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(start_dataloader):
        # 데이터를 디바이스로 이동
        inputs = inputs.to(device).permute(0,2,1,3,4)
        labels = labels.to(device)

        # 옵티마이저 초기화
        optimizer.zero_grad()
        
        # 모델 출력 계산
        outputs = model(inputs)
        
        # 손실 계산
        loss = criterion(outputs, labels)
        
        # 역전파 및 옵티마이저 업데이트
        loss.backward()
        optimizer.step()
        
        # 배치 손실 기록
        running_loss += loss.item()

    # 에포크별 평균 손실 출력
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(start_dataloader)}")

print("학습 완료")


Epoch 1, Loss: 0.6589767369112016
Epoch 2, Loss: 0.09484111151680663
Epoch 3, Loss: 0.039362522457536096
Epoch 4, Loss: 0.15697261568363977
Epoch 5, Loss: 0.036926437761572055
Epoch 6, Loss: 0.0026893180591736556
Epoch 7, Loss: 0.00017177184424213638
Epoch 8, Loss: 4.5169222213361055e-05
Epoch 9, Loss: 3.097920076161167e-05
Epoch 10, Loss: 2.3337759232437364e-05
학습 완료


In [6]:
def saveModel():
    torch.save(model.state_dict(), f'model/classifying_cnn.pt')
saveModel()